In [1]:
from urllib2 import urlopen
from bs4 import BeautifulSoup
import csv
import re

In [3]:
lvrList = ["http://www.trustile.com/products/lvr-series/lvr1000",
          "http://www.trustile.com/products/lvr-series/lvr2020",
          "http://www.trustile.com/products/lvr-series/lvr2060",
          "http://www.trustile.com/products/lvr-series/lvr3000"]

In [4]:
base_url = "http://www.trustile.com/"
def make_url(str):
    return base_url + str

In [5]:
def make_soup(url):
    html = urlopen(url).read()
    return BeautifulSoup(html)

In [7]:
def get_doorName(soup):
    return soup.h1.string

In [8]:
def get_pageFields(soup):
    return soup.findAll('fieldset')

In [6]:
def get_images(field):
    # get all types of images of this door model
    div = field.find('div',attrs={"class":"doorthumbs"})
    images = []
    for a in div.findAll('a'):
        aImage = {}
        aImage["type"] = a.get('alt')
        aImage["url"] = a.get('href')
        images.append(aImage)
    return images
            

In [9]:
# detail information on the right of the page
def get_details(field):
    div = field.find('div',attrs={"class":"doordetails"})
    details = {}
    for p in div.findAll('p'):
        try:
            ifDetail = p.get('class')[2]
            if ifDetail == "product-detail":           
                category = p.span
                if category.string == u'Series:\xa0':
                    details['Series'] = p.a.string
    
                if category.string == u'Application:\xa0':
                    stri = p.get_text()
                    app = re.search(u'(?<=\xa0).*',stri).group()
                    details['Application'] = app
    
                if category.string == u'Thickness:\xa0':
                    details['Thickness'] = p.findAll('span')[1].get_text()
    
                if category.string == u'Construction:\xa0':
                    try:
                        construction = []
                        for child in p.findAll('span')[1].children:
                            construction.append(child.string)
                    except:
                        constrction = re.search(u'(?<=\xa0).*',p.get_text()).group()
                    details['Construction'] = construction
                if category.string == u'Available Fire Rating:\xa0':
                    fireRating = []
                    for img in p.findAll('span')[1].findAll('img'):
                        aRating = {}
                        aRating["type"] = img.get('alt')
                        aRating["url"] = img.get('src')
                        fireRating.append(aRating)
                    details['Fire'] = fireRating
        except:
            print p.get_text()
    return details

In [10]:

def get_profiles(field):
    profiles = []
    for a in field.findAll('a'):
        profiles.append(a.get('title'))

    return profiles

In [16]:
# LVR series
# 
outFile = open("TrustileLVR_Doors.csv", "wb")
writer = csv.writer(outFile)
writer.writerow(["","Name","Images","Series","Application","Construction","Thickness","Fire Rating","Profiles Options","Other Options"])

for i in range(len(lvrList)):
    
    soup = make_soup(lvrList[i])
    
    name = get_doorName(soup)
    
    options = []
    fields = get_pageFields(soup)
    for field in fields:
        if field.legend.span.string == "Overview":
            images = get_images(field)
            
            details = get_details(field)
            try:
                fireRating = details["Fire"]
            except:
                fireRating = ""
        
        
        if field.legend.span.string == "Profiles":
#             print field.prettify()
            profiles = get_profiles(field)
#             print profiles
        
        # tab3: wood species
        if field.legend.span.string == "Wood Species":
            options.append('Wood')
    
        # tab4: glass options
        if field.legend.span.string == "Glass":
            options.append('Glass')
        
        # tab5: leather options
        if field.legend.span.string == "Leather":
            options.append('Leather')
    
        # tab6: Metal options
        if field.legend.span.string == "Metal":
            options.append('Metal')
    
#     print ([i+1, name, images, 
#                      details["Series"], details["Application"], details["Construction"], details["Thickness"], fireRating,
#                     options])
    writer.writerow([i+1, name, images, 
                     details["Series"], details["Application"], details["Construction"], details["Thickness"], fireRating,
                     profiles,
                     options])
outFile.close()

In [18]:
# profiles

soup = make_soup("http://www.trustile.com/products/profiles/louver-door-profiles")
table = soup.body.table
for a in table.findAll('a'):
    print a.get("title")
    print a.get("href")

Vented Louver Profile
http://www.trustile.com/sites/trustile.com/files/profiles/lvr-vented-louver-profile.png
False Louver Profile
http://www.trustile.com/sites/trustile.com/files/profiles/lvr-false-louver-profile_0.png


In [19]:
soup = make_soup("http://www.trustile.com/products/profiles/louver-door-profiles")
table = soup.body.table

profiles = open("TrustileLVR_profiles.csv","wb")
w = csv.writer(profiles)
w.writerow(["Type","Name","URL"])

for a in table.findAll('a'):
    w.writerow(["",a.get('title'),a.get('href')])
profiles.close()